In [ ]:
import opendatasets as od
od.download(dataset_id_or_url="https://www.kaggle.com/datasets/aslanahmedov/number-plate-detection",data_dir="C:\\Users\\amith\\Documents\\Datasets\\9 - Automatic Number Plate Recognition using ResNet and CNN")


In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from bs4 import *
import requests
import xml.etree.ElementTree as et
import os


In [78]:
xml_files_path = "C:\\Users\\amith\\Documents\\Datasets\\9 - Automatic Number Plate Recognition using ResNet and CNN\\number-plate-detection\\images"

list_of_xml_files = []

for i in os.listdir(xml_files_path):
    if ".xml" in i:
        list_of_xml_files.append(xml_files_path + "\\" + i)

for i in list_of_xml_files:
    element_tree = et.parse(i)
    element_root = element_tree.getroot()

    width = element_root.find("size").find("width").text
    height = element_root.find("size").find("height").text
    depth = element_root.find("size").find("depth").text

    print(width,height,depth)

    object = element_tree.find("object")

    x_min = object.find("bndbox").find("xmin").text
    x_max = object.find("bndbox").find("xmax").text
    y_min = object.find("bndbox").find("ymin").text
    y_max = object.find("bndbox").find("ymax").text

    print(x_min,x_max,y_min,y_max)


1920 1080 3
1093 1396 645 727
450 417 3
134 301 312 350
249 239 3
31 139 128 161
478 395 3
164 316 216 243
1800 1200 3
813 1067 665 724
377 244 3
66 154 166 197
500 289 3
360 434 174 195
610 485 3
137 262 249 290
441 363 3
207 356 174 287
484 456 3
184 342 220 257
550 534 3
148 239 250 320
275 183 3
131 187 130 144
586 331 3
183 249 211 227
730 563 3
80 239 364 402
500 344 3
179 347 256 290
400 300 3
39 108 129 157
250 166 3
15 200 33 126
600 337 3
157 316 226 278
602 339 3
120 249 161 202
1280 960 3
532 852 636 706
678 475 3
180 559 216 314
1920 1080 3
976 1284 669 754
800 485 3
647 736 332 387
802 557 3
311 506 324 370
600 398 3
342 494 243 288
250 250 3
11 246 78 165
531 512 3
165 328 276 324
600 337 3
149 399 269 327
505 343 3
24 142 209 245
500 399 3
58 221 198 271
774 642 3
301 504 351 402
594 379 3
419 513 173 213
500 291 3
198 349 107 143
503 439 3
111 320 133 286
2432 1368 3
795 1095 751 840
660 432 3
230 365 188 219
470 375 3
157 341 197 247
1632 918 3
787 910 473 513
1024 68